# Introduction to DuckDB, Polars & Parquet

In the previous session we connected to a remote **DB2** database using SQLAlchemy. That works, but every query travels over the network to the server and back. For analytical work — aggregations, joins, window functions — this round-trip overhead adds up quickly.

This session introduces three tools that form a modern local-analytics stack:

| Tool | What it is | Role in our workflow |
|------|-----------|---------------------|
| **Parquet** | A columnar file format | Stores our data on disk — compact, fast to read |
| **DuckDB** | An embedded analytical database | Runs SQL queries directly on Parquet files |
| **Polars** | A fast DataFrame library (alternative to pandas) | DataFrame operations when we want Python, not SQL |

### Why not just pandas?

| | pandas | Polars |
|---|---|---|
| **Engine** | Single-threaded, row-oriented | Multi-threaded, columnar (Apache Arrow) |
| **Speed on large data** | Slows down on millions of rows | Stays fast — designed for it |
| **Memory** | Often copies data | Zero-copy where possible |
| **Lazy evaluation** | No — every step executes immediately | Yes — can optimise a chain of operations before running |

### Our workflow

1. **Download** the tables from DB2 once (using SQLAlchemy)
2. **Save** each table as a Parquet file in a `data/` folder
3. **Query** locally with DuckDB or Polars — fast, offline, no server needed

---
## 1 — What is Parquet?

Parquet is a **columnar** file format, which means data is stored column-by-column instead of row-by-row. This makes analytical queries ("sum this column", "filter by that column") much faster because the engine only reads the columns it needs.

| Format | Structure | File size (typical) | Read speed for analytics |
|--------|-----------|--------------------|--------------------------|
| **CSV** | Row-based, plain text | Large | Slow — must parse every row and column |
| **Parquet** | Columnar, binary, compressed | Small (3–10x smaller than CSV) | Fast — reads only the columns needed |

Parquet files are the standard interchange format in modern data work. DuckDB, Polars, pandas, Spark, and most cloud tools all read them natively.

---
## 2 — Download DB2 Tables and Save as Parquet

This is a **one-time operation**. We use SQLAlchemy to read each table from DB2, then save it as a Parquet file. After this, you won't need the DB2 connection again.

### 2.1 — Connect to DB2 (same as session 00)

In [ ]:
import os
from pathlib import Path

import polars as pl
from dotenv import load_dotenv
from urllib.parse import quote_plus
from sqlalchemy import create_engine

load_dotenv()

engine = create_engine(
    f"db2+ibm_db://{os.getenv('DB_USERNAME')}:{quote_plus(os.getenv('DB_PASSWORD'))}"
    f"@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
)

print("DB2 connection ready.")

### 2.2 — Download all tables and save as Parquet

Polars can read directly from a database using `pl.read_database()`. We pass our SQL query and the SQLAlchemy engine — no pandas needed.

For each table we:
1. Read it from DB2 directly into a Polars DataFrame with `pl.read_database()`
2. Write to a `.parquet` file in the `data/` folder

> **Note:** The `TICKETS` table has ~35 million rows. Downloading it will take a few minutes depending on your connection. All other tables are small and finish in seconds.

In [ ]:
DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)

tables = [
    "COUNTRIES",
    "DEPARTMENT",
    "EMPLOYEE",
    "AIRPLANES",
    "AIRPORTS",
    "ROUTES",
    "FLIGHTS",
    "PASSENGERS",
    "TICKETS",
]

for table in tables:
    path = DATA_DIR / f"{table.lower()}.parquet"
    print(f"Downloading {table}...", end=" ", flush=True)

    # Polars reads directly from DB2 via the SQLAlchemy engine
    df = pl.read_database(f"SELECT * FROM IEPLANE.{table}", connection=engine)
    df.write_parquet(path)

    print(f"{len(df):>12,} rows  →  {path}")

print("\nDone! All tables saved as Parquet files in data/")

### 2.3 — Verify the files

In [ ]:
for path in sorted(DATA_DIR.glob("*.parquet")):
    size_mb = path.stat().st_size / (1024 * 1024)
    print(f"  {path.name:<25} {size_mb:>8.2f} MB")

---
## 3 — Querying Parquet Files with DuckDB

One of DuckDB's killer features: it can query Parquet files **directly** — no loading, no importing. Just point it at the file.

### 3.1 — Basic queries

In [ ]:
import duckdb

# Query a Parquet file directly — just use the file path as the table name
duckdb.sql("""
    SELECT *
    FROM 'data/airports.parquet'
    LIMIT 5
""")

In [ ]:
# How many flights per year?
duckdb.sql("""
    SELECT
        YEAR(departure)  AS year,
        COUNT(*)         AS num_flights
    FROM 'data/flights.parquet'
    GROUP BY year
    ORDER BY year
""")

In [ ]:
# Top 5 airports by number of departing routes
duckdb.sql("""
    SELECT
        a.airport,
        a.city,
        COUNT(r.route_code) AS num_routes
    FROM 'data/airports.parquet' a
    JOIN 'data/routes.parquet'   r ON a.iata_code = r.origin
    GROUP BY a.airport, a.city
    ORDER BY num_routes DESC
    LIMIT 5
""")

### 3.2 — Tip: Create VIEWs to avoid repeating file paths

Typing `'data/flights.parquet'` every time gets tedious. You can create **views** that act as aliases.

In [ ]:
# Create views for all our Parquet files
for path in sorted(DATA_DIR.glob("*.parquet")):
    name = path.stem  # e.g. "flights" from "flights.parquet"
    duckdb.execute(f"CREATE OR REPLACE VIEW {name} AS SELECT * FROM '{path}'")

# Now we can query by table name
duckdb.sql("SHOW TABLES")

In [ ]:
# Much cleaner!
duckdb.sql("""
    SELECT
        a.airport,
        a.city,
        COUNT(r.route_code) AS num_routes
    FROM airports a
    JOIN routes   r ON a.iata_code = r.origin
    GROUP BY a.airport, a.city
    ORDER BY num_routes DESC
    LIMIT 5
""")

### 3.3 — Getting results as a Polars DataFrame

Use `.pl()` at the end of a query to get the result as a Polars DataFrame. Use `.df()` if you need pandas instead.

In [ ]:
# .pl() returns a Polars DataFrame
revenue_by_class = duckdb.sql("""
    SELECT
        class,
        COUNT(*)                     AS num_tickets,
        ROUND(SUM(total_amount), 2)  AS total_revenue
    FROM tickets
    GROUP BY class
    ORDER BY total_revenue DESC
""").pl()

revenue_by_class

In [ ]:
type(revenue_by_class)

### 3.4 — DuckDB can also query Polars DataFrames directly

Just like with pandas, DuckDB can read a Polars DataFrame by its **Python variable name**. This works because both DuckDB and Polars use Apache Arrow under the hood.

In [ ]:
# Create a Polars DataFrame
sample = pl.DataFrame({
    "product": ["Laptop", "Phone", "Tablet", "Laptop", "Phone"],
    "revenue": [1200, 800, 450, 1350, 900],
})

# Query it with DuckDB — just use the variable name
duckdb.sql("""
    SELECT product, SUM(revenue) AS total
    FROM sample
    GROUP BY product
    ORDER BY total DESC
""")

---
## 4 — Working with Polars

Polars is a DataFrame library like pandas, but faster. It reads Parquet files natively.

### 4.1 — Reading Parquet files

In [ ]:
airports = pl.read_parquet("data/airports.parquet")
airports.head()

In [ ]:
# .schema shows column names and types
airports.schema

In [ ]:
# .describe() gives summary statistics — similar to pandas
airports.describe()

### 4.2 — Polars syntax crash course

Polars uses **expressions** instead of bracket indexing. The pattern is always:

```python
df.select(...)     # choose columns
df.filter(...)     # filter rows
df.group_by(...)   # group and aggregate
df.sort(...)       # order rows
```

Inside these methods, use `pl.col("column_name")` to refer to columns.

In [ ]:
employees = pl.read_parquet("data/employee.parquet")

# Select specific columns
employees.select("firstnme", "lastname", "salary").head()

In [ ]:
# Filter rows: employees earning more than 30,000
employees.filter(
    pl.col("salary") > 30_000
).select("firstnme", "lastname", "salary").head()

In [ ]:
# Group by + aggregate: average salary by education level
employees.group_by("edlevel").agg(
    pl.col("salary").mean().alias("avg_salary"),
    pl.col("salary").count().alias("num_employees"),
).sort("avg_salary", descending=True)

### 4.3 — Polars vs pandas cheat sheet

| Operation | pandas | Polars |
|-----------|--------|--------|
| Select columns | `df[["a", "b"]]` | `df.select("a", "b")` |
| Filter rows | `df[df["a"] > 5]` | `df.filter(pl.col("a") > 5)` |
| New column | `df["c"] = df["a"] + 1` | `df.with_columns((pl.col("a") + 1).alias("c"))` |
| Group + agg | `df.groupby("a").agg({"b": "sum"})` | `df.group_by("a").agg(pl.col("b").sum())` |
| Sort | `df.sort_values("a")` | `df.sort("a")` |
| Read Parquet | `pd.read_parquet("f.parquet")` | `pl.read_parquet("f.parquet")` |

---
## 5 — When to Use What

You now have three tools. Here's when each one shines:

| Tool | Best for | Example |
|------|----------|--------|
| **DuckDB SQL** | Multi-table joins, complex aggregations, exploring data | `SELECT ... FROM flights JOIN routes ...` |
| **Polars** | Step-by-step data transformations, feature engineering | `df.filter(...).with_columns(...).group_by(...)` |
| **DuckDB → `.pl()`** | SQL query, then continue in Polars | `duckdb.sql("...").pl().with_columns(...)` |

They work together seamlessly — use whichever feels more natural for each task.

### Example: a multi-table join in DuckDB

"For each passenger, show their most recent ticket with the flight origin and destination."

In [ ]:
duckdb.sql("""
    SELECT
        p.firstnme || ' ' || p.lastname  AS passenger,
        t.ticket_id,
        t.departure,
        r.origin,
        r.destination,
        t.total_amount
    FROM tickets t
    JOIN passengers p  ON t.passenger_id = p.id
    JOIN routes r      ON t.route_code   = r.route_code
    ORDER BY t.departure DESC
    LIMIT 10
""")

---
## 6 — Using Your Data in Future Sessions

Once the Parquet files exist in `data/`, this is all you need at the top of any notebook:

In [ ]:
# --- Copy this block into future notebooks ---
# import duckdb
# import polars as pl
# from pathlib import Path
#
# # Register all Parquet files as DuckDB views
# for path in sorted(Path("data").glob("*.parquet")):
#     duckdb.execute(f"CREATE OR REPLACE VIEW {path.stem} AS SELECT * FROM '{path}'")
#
# # Now query with SQL:
# duckdb.sql("SELECT COUNT(*) FROM tickets")
#
# # Or read directly into Polars:
# flights = pl.read_parquet("data/flights.parquet")

---
## 7 — Useful Commands Reference

### DuckDB

| Command | What it does |
|---------|-------------|
| `duckdb.sql("SHOW TABLES")` | List all views/tables |
| `duckdb.sql("DESCRIBE table")` | Show columns and types |
| `duckdb.sql("SUMMARIZE table")` | Quick statistics for every column |
| `.pl()` | Convert result to Polars DataFrame |
| `.df()` | Convert result to pandas DataFrame |

### Polars

| Command | What it does |
|---------|-------------|
| `df.head()` | First 5 rows |
| `df.schema` | Column names and types |
| `df.describe()` | Summary statistics |
| `df.shape` | (rows, columns) |
| `df.null_count()` | Count nulls per column |

In [ ]:
# SUMMARIZE gives a statistical overview — very handy for exploration
duckdb.sql("SUMMARIZE airports")

---
## 8 — Practice

Try these on your own. Use either DuckDB SQL or Polars — your choice! Solutions are at the bottom.

1. How many employees are there per department? Show department name and count, ordered by count descending.
2. What are the 5 longest routes by distance? Include origin and destination airport names.
3. What is the average ticket price per class (`E`, `P`, `B`)?
4. Which country has the most passengers?

In [ ]:
# Exercise 1: Employees per department


In [ ]:
# Exercise 2: 5 longest routes with airport names


In [ ]:
# Exercise 3: Average ticket price per class


In [ ]:
# Exercise 4: Country with most passengers


---
### Solutions (DuckDB SQL)

In [ ]:
# Solution 1
duckdb.sql("""
    SELECT d.deptname, COUNT(*) AS num_employees
    FROM employee e
    JOIN department d ON e.workdept = d.deptno
    GROUP BY d.deptname
    ORDER BY num_employees DESC
""")

In [ ]:
# Solution 2
duckdb.sql("""
    SELECT
        r.route_code,
        a1.airport AS origin_airport,
        a2.airport AS destination_airport,
        r.distance
    FROM routes r
    JOIN airports a1 ON r.origin      = a1.iata_code
    JOIN airports a2 ON r.destination  = a2.iata_code
    ORDER BY r.distance DESC
    LIMIT 5
""")

In [ ]:
# Solution 3
duckdb.sql("""
    SELECT
        class,
        ROUND(AVG(price), 2) AS avg_price
    FROM tickets
    GROUP BY class
    ORDER BY avg_price DESC
""")

In [ ]:
# Solution 4
duckdb.sql("""
    SELECT country, COUNT(*) AS num_passengers
    FROM passengers
    GROUP BY country
    ORDER BY num_passengers DESC
    LIMIT 10
""")

### Solutions (Polars)

In [ ]:
# Solution 1 — Polars
emp = pl.read_parquet("data/employee.parquet")
dept = pl.read_parquet("data/department.parquet")

emp.join(
    dept, left_on="workdept", right_on="deptno"
).group_by("deptname").agg(
    pl.len().alias("num_employees")
).sort("num_employees", descending=True)

In [ ]:
# Solution 3 — Polars
tix = pl.read_parquet("data/tickets.parquet")

tix.group_by("class").agg(
    pl.col("price").mean().round(2).alias("avg_price")
).sort("avg_price", descending=True)

In [ ]:
# Solution 4 — Polars
pax = pl.read_parquet("data/passengers.parquet")

pax.group_by("country").agg(
    pl.len().alias("num_passengers")
).sort("num_passengers", descending=True).head(10)